## Eager Execution

> Tensorflow's eager execution is an imperative programming environment that evalutes operations immediately, without building graphs: operations return concrete values instead of constructing a computational graph to run later.

In [1]:
# Set up eager mode.
import tensorflow as tf

tf.enable_eager_execution()

/home/tianqin/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Check whether it is running in eager mode.
tf.executing_eagerly()

True

In [4]:
x = [[2.]]
m = tf.matmul(x, x)
print("hello, {}".format(m))

hello, [[4.]]


As we can see, enabling eager execution changed how Tensorflow operations behave:

1. Now we do not create a Seession first and following by been executed in a graph, instead, we can execute it directly.

2. Operations are immediately executed and returned their vales to Python.

3. `tf.Tensor` objects reference concrete values instead of symbolic handles to nodes in a computational graph.

## Eager execution works nicely with Numpy

- Numpy operations accept `tf.Tensor` arguments.
- Tensorflow math operations convert Python objects and `tf.Tensor` objects.
- The `tf.Tensor.numpy` method returns the object's value as a Numpy `ndarray`


In [5]:
a = tf.constant([[1,2], [3,4]])
print(a)

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)


In [6]:
b = tf.add(a, 1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [7]:
print(a * b)  # Operator overloading


tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [9]:
# Numpy operators can accept tf.Tensor object directly
import numpy as np

c = np.multiply(a, b)
print(c)

[[ 2  6]
 [12 20]]


In [10]:
# Convert tf.Tensor object to numpy value.
print(a.numpy())

[[1 2]
 [3 4]]


The `tf.contrib.eager` module contains symbols available to both eager and graph execution environments and is useful for writing code to work with graphs.

In [11]:
tfe = tf.contrib.eager

## Building a model

When composing layers into models we can use `tf.keras.Sequential` to represent models which are a linear stack of layers.

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, input_shape=(784,)), # input is needed.
    tf.keras.layers.Dense(10)
])

Alternatively, organise models in classes by inheriting from `tf.keras.Model`.


In [15]:
class MNISTModel(tf.keras.Model):
    def __init__(self):
        super(MNISTModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(units = 10)
        self.dense2 = tf.keras.layers.Dense(units = 10)
    
    def call(self, input):
        """Run the model"""
        result = self.dense1(input)
        result = self.dense2(result)
        result = self.dense2(result)
        return result


## Eager training
-------------------------------------------------------
### Computing gradients

During eager execution, use `tf.GradientTape` to trace operations for computing gradients later.

A particular `tf.GradientTape` can only compute one gradient; subsequent calls throw a runtime error.

In [16]:
w = tfe.Variable([[1.0]])

with tf.GradientTape() as tape:
    loss = w * w

grad = tape.gradient(loss, [w])
print(grad)
    

[<tf.Tensor: id=78, shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>]


Example: use `tf.GraidentTape` to record forward-pass operations to train a simple model.

In [18]:
# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])

training_outputs = training_inputs * 3 + 2 + noise

def prediction(input, weight, bias):
    return input* weight + bias

# A loss functioin using mean-squared error
def loss(weights, biases):
    error = prediction(training_inputs, weights, biases) - training_outputs
    return tf.reduce_mean(tf.square(error))

# Return the derivative of loss with respect to weight and bias
def grad(weights, biases):
    with tf.GradientTape() as tape:
        loss_value = loss(weights, biases)
    return tape.gradient(loss_value, [weights, biases])
    
train_steps = 200
learning_rate = 0.01

# Start with arbitrary values for W and B
W = tfe.Variable(5.)
B = tfe.Variable(10.)

print("Initial loss: {:3f}".format(loss(W, B)))

for i in range(train_steps):
    dW, dB = grad(W, B)
    W.assign_sub(dW * learning_rate)
    B.assign_sub(dB * learning_rate)
    if i % 20 == 0:
        print("Loss at step {:3d} : {:3f}".format(i, loss(W, B)))

print("Final loss: {:3f}".format(loss(W, B)))
print("W = {}, B={}".format(W.numpy(), B.numpy()))

Initial loss: 69.130341
Loss at step   0 : 66.461044
Loss at step  20 : 30.433773
Loss at step  40 : 14.233822
Loss at step  60 : 6.947694
Loss at step  80 : 3.669947
Loss at step 100 : 2.195095
Loss at step 120 : 1.531334
Loss at step 140 : 1.232548
Loss at step 160 : 1.098025
Loss at step 180 : 1.037447
Final loss: 1.011073
W = 3.000185489654541, B=2.1008214950561523
